In [11]:
"""

- Fetch all of the Blog Post titles from WP website
- Generate Blog Post titles

Errors:
'code': 'rest_cannot_create' - Install "WordPress REST API Authentication" plugin

Start date: 24-07-2024
Last modification: 06-08-2024
"""

import requests
import json
import base64
import openai

def fetch_posts(per_page=100, max_pages=1000):
    session = requests.Session()
    session.headers.update(headers)
    all_titles = []

    for page in range(1, max_pages + 1):
        params = {
            "per_page": per_page,
            "page": page
        }
        
        response = session.get(WP_URL, params=params)
        
        if response.status_code != 200:
            print("Failed to fetch posts/EOL")
            print("Response:", response.json())
            break

        posts = response.json()
        
        if not posts:
            # If there are no posts returned, stop fetching further
            break
        
        for post in posts:
            post_title = post['title']['rendered']
            all_titles.append(post_title)

    return all_titles

def create_post(title, content, status, slug, excerpt, desciption, keywords):
    # Define the post data
    post_data = {
        "title": title,
        "content": content,
        "status": "publish",
        "slug": slug,
        "excerpt": excerpt,
        "meta": {
            "title": title,
            "description": desciption,
        },
        "yoast_meta": {
            "yoast_wpseo_title": title,
            "yoast_wpseo_metadesc": desciption,
            "yoast_wpseo_focuskw": title,
        },
        "tags": [1, 2, 3],
        "categories": [1, 2],  # Add relevant category IDs
    }

    # Send a POST request to create the post
    response = requests.post(WP_URL, headers=headers, data=json.dumps(post_data))

    if response.status_code == 201:
        print("Post created successfully!")
        print("Post ID:", response.json().get("id"))
    else:
        print("Failed to create post")
        print("Response:", response.json())

def generate_blog_title(topic, language, writing_style, writing_tone):
    # live_posts = fetch_posts()
    live_posts = []
    prompt = f"""Write a title (after "Title: ")for an article about {topic} in {language}. Title should include number. Style: {writing_style}. Tone: {writing_tone}. 
            Must be between 50 and 60 characters. Topic can be simmilar, but can not be the same to: {live_posts}.
            Based on genereted title create SEO optimalized: slug (after "Slug: "), excerpt (after "Excerpt: "), desciption (after "Desciption: "), 3 keywords (after "Keywords: ")"""
    # print(f"Prompt: {prompt}")
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=200 ,  # Adjust as needed
        n=1,
        stop=None,
        temperature=0.5 #Controls the randomness of the output. Higher values (e.g., 0.8) make the output more random
    )
    title, slug, excerpt, desciption, keywords = response.choices[0].text.strip().split('\n')

    title = title.split("Title: ")[1].replace('"', '')
    slug = slug.split("Slug: ")[1].replace('"', '')
    excerpt = excerpt.split("Excerpt: ")[1].replace('"', '')
    desciption = desciption.split("Desciption: ")[1].replace('"', '')
    keywords = keywords.split("Keywords: ")[1].replace('"', '')

    return title, slug, excerpt, desciption, keywords

def generate_blog_sections(title, language, writing_style, writing_tone, section_count):
    prompt = f"""Write {section_count} consecutive headings for an article about {title}, in {language}. 
            Style: {writing_style}. Tone: {writing_tone}. Each heading is between 40 and 60 characters. Use HTML for the headings <h2>"""
    # print(f"Prompt: {prompt}")
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=200 ,  # Adjust as needed
        n=1,
        stop=None,
        temperature=0.5 #Controls the randomness of the output. Higher values (e.g., 0.8) make the output more random
    )
    sections = response.choices[0].text.strip().split('\n')
    return sections

def generate_blog_content(title, language, writing_style, writing_tone, sections, paragraphs_per_section):
    prompt = f""""Write an article about "{title}" in {language}. The article is organized by the following headings:
            {sections} Write {paragraphs_per_section} paragraphs per heading. A paragraph should be maximum 110 words. Use HTML for formatting. Article should have around 700-800 words.
            Style: {writing_style}. Tone: {writing_tone}.""" #Add an introduction prefixed by "===INTRO: ", and a conclusion prefixed by "===OUTRO: ".
    # print(f"Prompt: {prompt}")
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=3800,  # Adjust as needed
        n=1,
        stop=None,
        temperature=0.5 #Controls the randomness of the output. Higher values (e.g., 0.8) make the output more random
    )
    return response.choices[0].text.strip()

# INSERT VARIABLES
openai.api_key = "YOUR_API_KEY"
WP_URL = "https://yourwebsite.pl/wp-json/wp/v2/posts"

# Base64 encode the username and password for authentication
credentials = f"LOGIN:PASSWORD"
token = base64.b64encode(credentials.encode())

headers = {
    "Authorization": f"Basic {token.decode('utf-8')}",
    "Content-Type": "application/json"
}

topic_for_title = "Kruszywo - jakie wybrać"
language = "Polish"
writing_style = "Informative"  # Creative, Narrative, Analtyical, Argumentative
writing_tone = "Neutral"  # Professional, Cheerful, Formal
section_count = "4-6" # Depending on the depth of the topic, 3-7 sections are generally good. Each section should have a heading (H2 or H3).
paragraphs_per_section = "3-4"  #  Aim for 2-4 paragraphs per section. Each paragraph should be around 100-150 words.

for _ in range(1):
    title, slug, excerpt, desciption, keywords = generate_blog_title(topic_for_title, language, writing_style, writing_tone)
    print(f"Title: {title}")
    sections = generate_blog_sections(title, language, writing_style, writing_tone, section_count)
    print(f"Sections: {sections}")
    article = generate_blog_content(title, language, writing_style, writing_tone, sections, paragraphs_per_section)
    print(f"Article: {article}")

    create_post(title, article, 'draft', slug, excerpt, desciption, keywords)


{
  "id": "cmpl-9ow8rWcIpq4gjNjv1GUVBHymUWccu",
  "object": "text_completion",
  "created": 1721926925,
  "model": "gpt-3.5-turbo-instruct",
  "choices": [
    {
      "text": "\nTitle: 5 rodzaj\u00f3w kruszywa - jakie wybra\u0107? \nSlug: 5-rodzajow-kruszywa-jakie-wybrac\nExcerpt: Poznaj 5 rodzaj\u00f3w kruszywa i dowiedz si\u0119, kt\u00f3re b\u0119dzie najlepsze dla Twojego projektu budowlanego.\nDesciption: Wyb\u00f3r odpowiedniego kruszywa jest kluczowy dla udanego projektu. Poznaj 5 rodzaj\u00f3w i wybierz to, kt\u00f3re spe\u0142ni Twoje wymagania.\nKeywords: kruszywo, rodzaje, wyb\u00f3r, projekt budowlany",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 115,
    "completion_tokens": 146,
    "total_tokens": 261
  }
}
Title: 5 rodzajów kruszywa - jakie wybrać? 
Sections: ['<h2> 1. Wprowadzenie do 5 rodzajów kruszywa </h2>', '<h2> 2. Kruszywo naturalne vs. sztuczne </h2>', '<h2> 3. Właściwości i zastosowania p

In [10]:
# Define the post data
keyword_phrase = title
post_data = {
    "title": title,
    "content": article,
    "status": "draft",
    "slug": slug,
    "excerpt": excerpt,
    "yoast_meta": {
        "yoast_wpseo_title": title,
        "yoast_wpseo_metadesc": desciption,
        "yoast_wpseo_focuskw": title,
    },
    "tags": [1, 2, 3],
    "categories": [1, 2]  # Add relevant category IDs
}

# Send a POST request to create the post
response = requests.post(WP_URL, headers=headers, data=json.dumps(post_data))

if response.status_code == 201:
    print("Post created successfully!")
    print("Post ID:", response.json().get("id"))
else:
    print("Failed to create post")
    print("Response:", response.json())

Post created successfully!
Post ID: 74


In [63]:
import requests

# Fetch a post to inspect its fields
# response = requests.get()  # Replace with an actual post ID
response = requests.get('https://bazakruszywa.pl/wp-json/wp/v2/posts/56', headers=headers, data=json.dumps(post_data))


# Check if the request was successful
if response.status_code == 200:
    # Parse and pretty-print the JSON response
    response_json = response.json()
    print(json.dumps(response_json, indent=4))
else:
    print(f"Error: {response.status_code}")
    print(response.text)

{
    "id": 56,
    "date": "2024-07-25T13:40:58",
    "date_gmt": "2024-07-25T11:40:58",
    "guid": {
        "rendered": "https://bazakruszywa.pl/?p=56"
    },
    "modified": "2024-07-25T13:40:58",
    "modified_gmt": "2024-07-25T11:40:58",
    "slug": "5-rodzajow-kruszywa-jakie-wybrac-2",
    "status": "draft",
    "type": "post",
    "link": "https://bazakruszywa.pl/?p=56",
    "title": {
        "rendered": "5 rodzaj\u00f3w kruszywa &#8211; jakie wybra\u0107"
    },
    "content": {
        "rendered": "<h2> 1. Rodzaje kruszywa &#8211; przegl\u0105d popularnych opcji </h2>\n<p>Kruszywo jest nieod\u0142\u0105cznym elementem budowlanym, wykorzystywanym w wielu projektach. Jego g\u0142\u00f3wnym zadaniem jest wzmocnienie pod\u0142o\u017ca oraz utrzymanie stabilno\u015bci konstrukcji. Istnieje wiele rodzaj\u00f3w kruszywa, kt\u00f3re r\u00f3\u017cni\u0105 si\u0119 mi\u0119dzy sob\u0105 sk\u0142adem, wytrzyma\u0142o\u015bci\u0105 i zastosowaniem. Najpopularniejszymi opcjami s\u0105 p